# 模型层

# 聚合查询

只要是跟数据库线管的模块基本上都在**django.db.models**里面，如果没有就是在**django.db**中

聚合查询通常情况下都是配合分组一起使用。

模块导入：
`from django.db.models import Max, Min, Sum, Count, Avg`


1. 所有书的评价价格

```
res = models.Book.objects.aggregate(Avg('price'))
```

2. 上述方法一次性使用

```
res = models.Book.objects.aggregate(Avg('price'), Max('prince'))
```

# 分组查询

1. 统计每一本书的作者个数

```
res = models.Book.objects.annotate()  # models后面点什么,就是按什么分组,此时按Book分组

res = models.Book.objects.annotate(author_num=Count('authors')).values('title', 'author_num')

# author_num 是自己定义的字段,用来存储统计出来的额每本书对应的作者个数
```

2. 统计每个出版社卖的最便宜的书的价格

```
res = models.Publish.objects.annotate(min_price=Min('book__price')).values('name', 'min_price')
```

3. 统计不止一个作者的图书

```
# 1.先按照图书分组,求每一本书对应作者个数
# 2.过滤出不止一个作者的图书

res = models.Book.objects.annotate(author_count=Count('authors')).filter(author_count__gt=1).values('name', 'author_count')
```

**只要你的orm语句得出的结果还是一个queryset对象,那么它就可以继续无限制的点queryset对象封装的方法**

按照指定的字段分组:

`models.Book.objects.valus('price').annotate()`

# F与Q查询

## F

1. 查询卖出数大于库存数的书籍

```
F 查询,能够帮助你直接获取到表中的某个字段对应的数据

from django.db.models import F
res = models.Book.objects.filter(maichu__gt=F('kucun'))
```

2. 将所有书记的价格提升500块

```
models.Book.objects.update(price=F('price') + 500)
```

3. 将所有书的名称后面加上爆款两个字

```
在操作字符类型的数据时,F不能够直接做到字符串的拼接

from django.db.models.functions import Concat
from django.db.models import Value
models.Book.objects.update(title=Concat(F('title'), Value('爆款')))
```

## Q

1. 查询卖出数大于100或者价格小于600的书籍

```
from django.db.models import Q
res = models.Book.objects.filter(Q(maichu__gt=100)|Q(price_lt=600))  # | 是 or关系,使用,分割还是and关系

res = models.Book.objects.filter(~Q(maichu__gt=100)|Q(price_lt=600))  # ~ 是 not关系
```

Q 的高阶用法,能够将查询条件的左边也变成字符串的形式

```
q = Q()
q.children.append(('maichu_gt', 100))
q.children.append(('price_lt', 600))

res = models.Book.objects.filter(q)  # 默认是and关系

修改链接关系:
q.connector = 'or'
```

# django 中开启事务

目前只需要掌握django中如何简单的开启事务。

```
from django.db import transaction
with transaction.atomic():
    # 此处写 书写的所有orm操作都是属于同一个事务
```

# orm 常用字段以及参数

1. AutoField

    主键字段 必须设置primaty_key=True
    
2. CharField  对应到数据库是 varchar

    verbose_name  字段的注释
    
    max_length  长度
    
3. Integerfield  对应 int

4. DecimalField

    max_digits
    
    decimal_places  小数长度
    
5. EmailFiled  对应 varchar(254)

6. DateField  对应 date

7. DateTimeField  对应 datetime

    auto_now  每次修改数据时自动更新当前时间
    
    auto_now_add 只在创建数据的时候记录创建时间后续不会自动更新
    
8. BooleanField  对应 布尔值类型

    该字段传布尔值（False/True）  数据库里面存（0/1）
    
9. TextField

    该字段可以用来存大段内容（文章、博客），无字数限制
    
10. FileField

    upload_to = '/data'  给该字段传递一个文件对象，会自动将文件保存到/data目录下，然后将文件路径保存到数据库中/data/a.txt

## django 支持自定义字段

```
class MyCharField(models.Field):
    def __init__(self, max_length, *args, **kwargs):
        self.max_length = max_length
        # 调用父类的init方法
        super().__init__(max_length=max_length, *args, **kwargs)
        
    def db_type(self, connection):
        # 返回真正的数据类型以及各种约束条件，是数据库中的类型
        return 'char(%s)'%self.max_length
        
        
# 使用自定义字段
myfield = MyCharField(max_length=16, null=True)
```

# 数据库查询优化

orm语句的特点：

    惰性查询，如果仅仅书写了orm语句，在后面根本没有用到该语句所查询出来的参数，那么orm会自动识别，直接不执行。

## only与defer

1. 实现一个获取到的是一个数据对象 ，然后点title就能拿到书名，并且没有其他字段

```
res = models.Book.objects.only('title')
for i in res:
    print(i.title)  # 点only括号内的字段，不会走数据库
    print(i.price)  # 点only括号内没有的字段，会重新走数据库查询而all不需要走了
```

defer与only刚好相反：

    defer括号内放的字段不在查询出来的对象里面，查询该字段需要重新走数据库，而如果查询的是非括号内的字段，则不需要走数据库了
    
```
res = models.Book.ojects.defer('title') # 对象除了没有title属性之外其他的都有
```

## select_related 与 prefetch_related  跟跨表操作有关

```
res = models.Book.objects.all()
for i in res:
    print(i.publish.name)  # 每循环一次就要走一次数据库查询
    
    
res = models.Book.objects.select_related('author')
"""
select_related内部直接先将book与publish连接起来，然后一次性将大表里面的所有数据全部封装给查询出来的对象，这个时候对象无论是点book表的数据还是publish表数据都无需再走数据库了

select_related括号内只能放外键字段（一对一，一对多），多对多是不行的。
"""


res = models.Book.objects.prefetch_related('publish') # 子查询
# 内部方法其实是子查询，但是在使用上的感觉两者是一样的
```

作业：数据库三大设计范式整理